In [ ]:
import boto3

In [ ]:
## Can we find the 'droids we're looking for?
def get_id_for_db(client,name):
    id = 'not-found'
    response = client.describe_db_instances()
    instances = response['DBInstances']
    for i in instances:
        if i['DBName'] == name.upper():
            id = i['DBInstanceIdentifier']
            break

    if id == 'not-found':
        raise ValueError('Database ' + db_name + ' not found in region ' + source_region)
    else:
        return id

In [ ]:
def get_source_snapshots(client,dbid):
    response = client.describe_db_snapshots(
         DBInstanceIdentifier=dbid
    )

    source_snapshots = []
    snapshots = response['DBSnapshots']
    for s in snapshots:
        if s['Status'] != 'available':
            continue
        source = {}
        source['id'] = s['DBSnapshotIdentifier']
        source['arn'] = s['DBSnapshotArn']
        source['state'] = s['Status']
        source_snapshots.append(source)

    return source_snapshots



In [ ]:
def get_dest_snapshots(client,dbid):
    response = client.describe_db_snapshots(
         DBInstanceIdentifier=dbid
    )

    dest_snapshots = []
    snapshots = response['DBSnapshots']
    for s in snapshots:
        dest_snapshots.append(s['DBSnapshotIdentifier'])

    return dest_snapshots



In [ ]:
def get_snaps_to_copy(src_snaps,dest_snaps):
    copy = []
    for ss in src_snaps:
        idMinusRDS = ss['id'].split(':')[1]
        if idMinusRDS not in dest_snaps:
            copy.append(ss)
            
    return copy
    




In [ ]:
targetParts = TargetDBSnapshotIdentifier=source_snapshots[0]['id'].split(':')
target = targetParts[1]
response = dest_client.copy_db_snapshot(
    SourceDBSnapshotIdentifier=source_snapshots[0]['arn'],
    TargetDBSnapshotIdentifier=target
)

print response

In [67]:
## Setup

import boto3

source_region = 'us-west-2'
dest_region = 'us-east-1'
db_name = 'dsdb'

source_client = boto3.client('rds',region_name=source_region)
dest_client = boto3.client('rds',region_name=dest_region)

id = get_id_for_db(source_client,db_name)
print id

snaps = get_source_snapshots(source_client,id)
print snaps

desc_snapshots = get_dest_snapshots(dest_client, id)
print desc_snapshots

s2c = get_snaps_to_copy(source_snapshots, dest_snapshots)

for s in s2c:
    target = s['id'].split(':')[1]
    print target

dm1l1y5i6j2dqai
[{'state': 'available', 'id': 'rds:dm1l1y5i6j2dqai-2017-04-05-14-13', 'arn': 'arn:aws:rds:us-west-2:444472610063:snapshot:rds:dm1l1y5i6j2dqai-2017-04-05-14-13'}, {'state': 'available', 'id': 'rds:dm1l1y5i6j2dqai-2017-04-06-06-06', 'arn': 'arn:aws:rds:us-west-2:444472610063:snapshot:rds:dm1l1y5i6j2dqai-2017-04-06-06-06'}]
['dm1l1y5i6j2dqai-2017-04-05-14-13']
dm1l1y5i6j2dqai-2017-04-06-06-06
